In [19]:
@inline isnum(v) = isbits(v) && typeof(v) != Date && typeof(v) != DateTime

# Custom JSON generator
function jsonify(sasobj, names, row) 
    s = IOBuffer()
    i = 0
    print(s,"{")
    for k in names
        i > 0 && print(s, ",")
        print(s, "\"", k, "\":")
        v = sasobj[:data][k][row]
        !isnum(v) && print(s, "\"")
        print(s, v)
        !isnum(v) && print(s, "\"")
        i += 1
    end
    print(s,"}")
    String(take!(s))
end

# Buffered version (non-allocating)
function jsonify(s::IOBuffer, sasobj, names, row) 
    i = 0
    print(s,"{")
    for k in names
        i > 0 && print(s, ",")
        print(s, "\"", k, "\":")
        v = sasobj[:data][k][row]
        !isnum(v) && print(s, "\"")
        print(s, v)
        !isnum(v) && print(s, "\"")
        i += 1
    end
    print(s,"}")
    String(take!(s))
end

jsonify (generic function with 2 methods)

In [20]:
using BenchmarkTools

foo(x) = json(Dict([(k,x[:data][k][1]) for k in x[:column_symbols]]))
bar(x) = jsonify(x, x[:column_symbols], 1)
bar2(x, s) = jsonify(s, x, x[:column_symbols], 1)

bar2 (generic function with 1 method)

In [21]:
using SASLib 
x = readsas("/Users/tomkwong/iCloudDocs/Programming/Julia/SASLib/test/data_pandas/productsales.sas7bdat")

Read /Users/tomkwong/iCloudDocs/Programming/Julia/SASLib/test/data_pandas/productsales.sas7bdat with size 1440 x 10 in 0.00747 seconds


Dict{Symbol,Any} with 19 entries:
  :filename             => "/Users/tomkwong/iCloudDocs/Programming/Julia/SASLib…
  :page_length          => 8192
  :file_encoding        => "US-ASCII"
  :system_endianness    => :LittleEndian
  :compression          => "None"
  :ncols                => 10
  :column_types         => Type[Float64, Float64, String, String, String, Strin…
  :column_info          => Tuple{Int64,Symbol,Symbol,Type,DataType}[(1, :ACTUAL…
  :data                 => Dict{Any,Any}(Pair{Any,Any}(:QUARTER, [1.0, 1.0, 1.0…
  :perf_type_conversion => 0.000149216
  :page_count           => 18
  :column_names         => String["ACTUAL", "PREDICT", "COUNTRY", "REGION", "DI…
  :column_symbols       => Symbol[:ACTUAL, :PREDICT, :COUNTRY, :REGION, :DIVISI…
  :column_lengths       => [8, 8, 10, 10, 10, 10, 10, 8, 8, 8]
  :file_endianness      => :LittleEndian
  :process_id           => 1
  :nrows                => 1440
  :perf_read_data       => 0.00702284
  :column_offsets       => [0, 8,

In [22]:
using JSON
foo(x)

"{\"QUARTER\":1.0,\"YEAR\":1993.0,\"COUNTRY\":\"CANADA\",\"DIVISION\":\"EDUCATION\",\"REGION\":\"EAST\",\"MONTH\":\"1993-01-01\",\"PREDICT\":850.0,\"ACTUAL\":925.0,\"PRODTYPE\":\"FURNITURE\",\"PRODUCT\":\"SOFA\"}"

In [23]:
bar(x)

"{\"ACTUAL\":925.0,\"PREDICT\":850.0,\"COUNTRY\":\"CANADA\",\"REGION\":\"EAST\",\"DIVISION\":\"EDUCATION\",\"PRODTYPE\":\"FURNITURE\",\"PRODUCT\":\"SOFA\",\"QUARTER\":1.0,\"YEAR\":1993.0,\"MONTH\":\"1993-01-01\"}"

In [24]:
buf = IOBuffer()
bar2(x, buf)

"{\"ACTUAL\":925.0,\"PREDICT\":850.0,\"COUNTRY\":\"CANADA\",\"REGION\":\"EAST\",\"DIVISION\":\"EDUCATION\",\"PRODTYPE\":\"FURNITURE\",\"PRODUCT\":\"SOFA\",\"QUARTER\":1.0,\"YEAR\":1993.0,\"MONTH\":\"1993-01-01\"}"

In [25]:
@benchmark foo($x)

BenchmarkTools.Trial: 
  memory estimate:  3.55 KiB
  allocs estimate:  81
  --------------
  minimum time:     32.952 μs (0.00% GC)
  median time:      33.991 μs (0.00% GC)
  mean time:        35.332 μs (0.91% GC)
  maximum time:     3.343 ms (96.62% GC)
  --------------
  samples:          10000
  evals/sample:     1

In [26]:
@benchmark bar($x)

BenchmarkTools.Trial: 
  memory estimate:  1.20 KiB
  allocs estimate:  18
  --------------
  minimum time:     7.934 μs (0.00% GC)
  median time:      8.066 μs (0.00% GC)
  mean time:        8.373 μs (1.57% GC)
  maximum time:     715.085 μs (95.48% GC)
  --------------
  samples:          10000
  evals/sample:     4

In [27]:
@benchmark bar2($x, $buf)

BenchmarkTools.Trial: 
  memory estimate:  1.19 KiB
  allocs estimate:  19
  --------------
  minimum time:     8.182 μs (0.00% GC)
  median time:      8.321 μs (0.00% GC)
  mean time:        8.458 μs (1.10% GC)
  maximum time:     969.681 μs (96.33% GC)
  --------------
  samples:          10000
  evals/sample:     3